# 使用 cvxpy 优化高级投资组合问题

## 安装 cvxpy 及其他库

In [ ]:
import sys
!{sys.executable} -m pip install -r requirements.txt

## 导入

In [ ]:
import cvxpy as cvx
import numpy as np
import quiz_tests_advanced

## 我们的目标是什么？
http://www.cvxpy.org/

下面看看如何通过优化方法实现更高级的目标。我们希望最小化投资组合方差，并且希望紧密跟踪一个市值加权指数。也就是说，我们想最小化投资组合权重与指数权重之间的距离。

$Minimize \left [ \sigma^2_p + \lambda \sqrt{\sum_{1}^{m}(weight_i - indexWeight_i)^2} \right  ]$，其中 $m$ 是投资组合中的股票数量，$\lambda$ 是你可以设置的比例因子。


## 提示

### x 向量
为了创建包含 M 个变量的向量 $\mathbf{x} = \begin{bmatrix}
x_1 &...& x_M
\end{bmatrix}
$
我们可以使用 `cvx.Variable(m)`

### 协方差矩阵
如果有 $m$ 个股票，协方差矩阵是一个 $m \times m$ 矩阵，包含每对股票之间的协方差。我们可以使用 [numpy.cov](https://docs.scipy.org/doc/numpy/reference/generated/numpy.cov.html) 获得协方差。向其传入一个二维数组，行表示股票，列表示同一时期的观察值。

协方差矩阵 $\mathbf{P} = 
\begin{bmatrix}
\sigma^2_{1,1} & ... & \sigma^2_{1,m} \\ 
... & ... & ...\\
\sigma_{m,1} & ... & \sigma^2_{m,m}  \\
\end{bmatrix}$

### 投资组合方差
可以将投资组合方差表示成 $\sigma^2_p = \mathbf{x^T} \mathbf{P} \mathbf{x}$

$\mathbf{x^T} \mathbf{P} \mathbf{x}$ 称为二次型。
我们可以使用 cvxpy 函数 `quad_form(x,P)` 获得二次型。

### 与指数权重之间的距离
我们希望获得能紧密跟踪指数的投资组合权重。所以我们希望最小化二者之间的距离。
根据勾股定理，在 x,y 平面中的两点之间的距离等于 x 和 y 距离的平方和再求平方根。将此定理延伸到任何维数后变成 L2 范数。所以：$\sqrt{\sum_{1}^{n}(weight_i - indexWeight_i)^2}$ ， 还可以写成 $\left \| \mathbf{x} - \mathbf{index} \right \|_2$。有一个 cvxpy 函数，叫做 [norm()](https://www.cvxpy.org/api_reference/cvxpy.atoms.other_atoms.html#norm)
`norm(x, p=2, axis=None)`。默认情况下就是计算 L2 范数，所以需要传入一个参数，即投资组合权重与指数权重之间的差异。

### 目标函数
我们想要最小化投资组合方差以及投资组合权重与指数权重之间的距离。
还需要选择一个 `scale` 常量，即表达式中的 $\lambda$。它表示相对于最小化投资组合方差而言，最小化与指数之间差异的优先级是多少。如果 `scale` ($\lambda$) 的值很大，你认为最小化差异的优先级更高，还是最小化方差的优先级更高？

我们可以使用 cvxpy  `objective = cvx.Minimize()` 得出目标函数。你认为应该向此函数传入什么？

### 约束条件
还可以使用列表定义约束条件。例如，如果你希望权重之和为 1，则 $\sum_{1}^{n}x = 1$。并且可能需要只做多头寸，也就是不做空，所以权重不能为负。所以对所有 $i$ 来说，$x_i >0 $，可以将变量另存为 `[x >= 0, sum(x) == 1]`，其中 x 是用 `cvx.Variable()` 创建的。

### 优化
设定目标函数和约束条件后，我们可以求解 $\mathbf{x}$ 的值。
cvxpy 具有构造函数 `Problem(objective, constraints)`，它返回一个 `Problem` 对象。

`Problem` 对象具有函数 solve()，它返回解的最小值，即投资组合的最小方差。

它还会更新向量 $\mathbf{x}$。

我们可以使用 `x.value` 查看得出最小投资组合方差的 $x_A$ 和 $x_B$ 值。

## 小测验

In [ ]:
import cvxpy as cvx
import numpy as np

def optimize_portfolio(returns, index_weights, scale=.00001):
    """
    Create a function that takes the return series of a set of stocks, the index weights,
    and scaling factor. The function will minimize a combination of the portfolio variance
    and the distance of its weights from the index weights.  
    The optimization will be constrained to be long only, and the weights should sum to one.
    
    Parameters
    ----------
    returns : numpy.ndarray
        2D array containing stock return series in each row.
        
    index_weights : numpy.ndarray
        1D numpy array containing weights of the index.
        
    scale : float
        The scaling factor applied to the distance between portfolio and index weights
        
    Returns
    -------
    x : np.ndarray
        A numpy ndarray containing the weights of the stocks in the optimized portfolio
    """
    # TODO: Use cvxpy to determine the weights on the assets
    # that minimizes the combination of portfolio variance and distance from index weights
    
    # number of stocks m is number of rows of returns, and also number of index weights
    #m = 
    
    #covariance matrix of returns
    #cov = 
    
    # x variables (to be found with optimization)
    #x = 
    
    #portfolio variance, in quadratic form
    #portfolio_variance = 
    
    # euclidean distance (L2 norm) between portfolio and index weights
    #distance_to_index = 
    
    #objective function
    #objective = 
    
    #constraints
    #constraints = 

    #use cvxpy to solve the objective
    
    #retrieve the weights of the optimized portfolio
    #x_values
    
    return x_values

quiz_tests_advanced.test_optimize_portfolio(optimize_portfolio)

In [ ]:
"""Test with a 3 simulated stock return series"""
days_per_year = 252
years = 3
total_days = days_per_year * years

return_market = np.random.normal(loc=0.05, scale=0.3, size=days_per_year)
return_1 = np.random.uniform(low=-0.000001, high=.000001, size=days_per_year) + return_market
return_2 = np.random.uniform(low=-0.000001, high=.000001, size=days_per_year) + return_market
return_3 = np.random.uniform(low=-0.000001, high=.000001, size=days_per_year) + return_market
returns = np.array([return_1, return_2, return_3])

"""simulate index weights"""
index_weights = np.array([0.9,0.15,0.05])

"""try out your optimization function"""
x_values = optimize_portfolio(returns, index_weights, scale=.00001)

print(f"The optimized weights are {x_values}, which sum to {sum(x_values):.2f}")

如果你遇到问题，请在[此处](m3l4_cvxpy_advanced_solution.ipynb)查看解答